In [1]:
input_data = [
    {
        "target_symbol": "PIK3CA",
        "target_id": "ENSG00000121879",
        "diseases": [
            {
                "efo_label": "breast carcinoma",
                "efo_id": "EFO_0000305"
            },
            {
                "efo_label": "colorectal cancer",
                "efo_id": "EFO_1001951",
            },
        ]
    },
    {
        "target_symbol": "AKT1",
        "target_id": "ENSG00000142208",
        "diseases": [
            {
                "efo_label": "breast carcinoma",
                "efo_id": "EFO_0000305"
            }
        ]
    },
    {
        "target_symbol": "APC",
        "target_id": "ENSG00000134982",
        "diseases": [
            {
                "efo_label": "colorectal cancer",
                "efo_id": "EFO_1001951",
            }
        ]
    }
]

In [3]:
for entry in input_data:
    print(entry["target_symbol"])

PIK3CA
AKT1
APC


In [5]:
import pandas
import numpy
import requests
import json

In [18]:
url = "https://platform-api.opentargets.io/v3/platform/private/target/ENSG00000134982"
raw_api_response = requests.get(url).json()
pretty_api_response = json.dumps(raw_api_response, indent=2, sort_keys=True)
print(pretty_api_response)

{
  "alias_name": [],
  "alias_symbol": [],
  "approved_name": "APC regulator of WNT signaling pathway",
  "approved_symbol": "APC",
  "biotype": "protein_coding",
  "cancerbiomarkers": [
    {
      "association": "Responsive",
      "biomarker": "APC oncogenic mutation",
      "diseases": [
        {
          "id": "EFO_0000365",
          "label": "Colorectal adenocarcinoma"
        }
      ],
      "drug": "[]",
      "drugfamily": "[Tankyrase inhibitor]",
      "drugfullname": "Tankyrase inhibitors",
      "evidencelevel": "Pre-clinical",
      "gene": "APC",
      "individualbiomarker": "",
      "references": {
        "other": [],
        "pubmed": [
          {
            "pmid": "22440753"
          },
          {
            "pmid": "23539443"
          }
        ]
      }
    }
  ],
  "ccds_ids": [],
  "chembl": [],
  "chromosome": "5",
  "cytobands": "",
  "dbxrefs": [
    "Bgee:ENSG00000134982",
    "BindingDB:P25054",
    "BioCyc:MetaCyc:ENSG00000134982-MONOMER",
    "

In [19]:
print(raw_api_response["biotype"])

protein_coding


In [24]:
important_fields = [
    "approved_name",
    "biotype",
    "uniprot_id"
]

all_data = []

data_row = []

for field in important_fields:
    data_row.append(raw_api_response[field])

all_data.append(data_row)

print(all_data)

[['APC regulator of WNT signaling pathway', 'protein_coding', 'P25054']]


In [30]:
df = pandas.DataFrame(all_data)
df.columns = important_fields
df.head()

,approved_name,biotype,uniprot_id
0,APC regulator of WNT signaling pathway,protein_coding,P25054
